## Import các thư viện

In [131]:
using Pkg
using CSV
using DataFrames
using StatsBase
using Random

## Hàm chia tập dữ liệu thành training set và test set

In [115]:
function splitdf(df, pct)
        @assert 0 <= pct <= 1
        ids = collect(axes(df, 1))
        shuffle!(ids)
        sel = ids .<= nrow(df) .* pct
        return view(df, sel, :), view(df, .!sel, :)
end

splitdf (generic function with 1 method)

## Hàm regulazation_att
***Dùng để rời rạc hoá một thuộc tính***
## Input:
* Dataframe
* Tên thuộc tính
* Khoảng cần thiết (dạng tuple (a,b)), tất cả các giá trị i (a<=i<=b) được dán nhãn 1, các giá trị còn lại nhãn 2
## Output:
* Dataframe sau khi chuẩn hoá

In [116]:
function regulazation_att(df,att,interval)
    row = size(df)[1]
    new = copy(df)
    for i in 1:row
        val = new[:,att][i]
        if (val>=interval[1] && val<=interval[2])
            new[i,att] = 1
        else
            new[i,att] = 2
        end
    end
    return new
end

regulazation_att (generic function with 1 method)

## Hàm regulazation
***Dùng để rời rạc hoá tất cả các thuộc tính***
## Input:
* Dataframe
##  Output:
* Dataframe sau khi chuẩn hoá

In [117]:
# Roi rac hoa cac thuoc tinh
function regulazation(df)
    new = regulazation_att(df,"sepal.length",(7.2,8))
    new = regulazation_att(new,"sepal.width",(2,2.5))
    new = regulazation_att(new,"petal.length",(1,2))
    new = regulazation_att(new,"petal.width",(1.9,3))
    return new
end

regulazation (generic function with 1 method)

## Hàm Entropy
***Tính độ đo entropy của một thuộc tính (attribute)***
## Input
* df: Dataframe
* att: Tên thuôc tính
## Output
* entropy: độ đo entropy

In [118]:
# Tinh Entropy cua mot thuoc tinh
function Entropy(df,att)
    variety = ["Setosa","Versicolor","Virginica"]
    size_df = size(df)[1]
    entropy = 0
    for i in unique(df[:,att])
        entropy_class = 0
        filter = df[:,att].== i
        df_of_class = df[filter,["variety"]]
        size_class = size(df_of_class)[1]
        for name in variety
            filter_variety = df_of_class[:,"variety"].== name
            value = size(df_of_class[filter_variety,:])[1]/size_class
            if(value != 0)
                value = -value*(log(value))
                entropy_class+=value
            end
        end
        entropy+=(size_class/size_df)*entropy_class
    end
    return entropy
end

Entropy (generic function with 1 method)

## Hàm sort_by_entropy
***Dùng để sắp xếp các thuộc tính giảm dần entropy***
## Input: 
* df: Dataframe đã chuẩn hoá
## Output:
* result: array tên các thuộc tính đã sắp xếp

In [119]:
function sort_by_entropy(df)
    att = names(df)
    att = att[1:(length(att)-1)]
    dict = Dict()
    for i in att
        entropy = Entropy(df,i)
        dict[i] = entropy
    end
    sorted = (sort(collect(dict), by=x->x[2], rev=true))
    result =[ i[1] for i in sorted]
    return result
end

sort_by_entropy (generic function with 1 method)

In [120]:
function count_variety(df)
    variety = df[:,"variety"]
    dict = Dict()
    for i in unique(variety)
        count = 0
        for j in 1:size(variety)[1]
            if(variety[j] == i)
                count+=1
            end
        end
        dict[i] = count
    end
    return dict
end

count_variety (generic function with 1 method)

## Hàm checked_classified
***Kiểm tra xem với luật vừa tìm được có thể phần lớp hoàn toàn về một lớp không. Hàm này dùng luật để lọc ra các dòng thoả và kiểm tra attribute "variety" chỉ có một nhãn duy nhất hay không.***
## Input:
* data: Dataframe đã chuẩn hoá
* dict_rules (dictionary): luật
## Output:
* Nếu đã thuộc về duy nhất một nhãn trả về tên nhãn đó
* Nếu không trả về nothing

In [121]:
function check_classified(dict_rules, data)
    new = copy(data)
    variety = ["Setosa","Versicolor","Virginica"]
    for (i,j) in dict_rules
        filter = new[:,i].==j
        new = new[filter,:]
    end
    num_class = length(unique(new[:,"variety"]))
    if (num_class == 1)
        return unique(new[:,"variety"])[1]
    end
    return nothing
end

check_classified (generic function with 1 method)

## Hàm decision_tree
***Thuật toán ID3 để tìm ra các luật trong tập dữ liệu***
## Input:
* training_set: tập training sau khi đã chuẩn hoá
* attribute: tập các thuộc tính sau khi đã sắp xếp giảm dần theo entropy
* parent: node cha (được khởi tạo rỗng ban đầu)
* result: kết quả (các luật)
## Output:
* result: các luật

In [122]:
function decision_tree(training_set,attribute,parent,result)
    if(attribute==[])
        return 
    end
    data = copy(training_set)
    att = pop!(attribute)
    for i in unique(data[:,att])
        child_node = copy(parent)
        child_node[att] =i
        class = check_classified(child_node,training_set)
        if(class != nothing)
            rule = copy(child_node)
            rule["variety"] = class
            push!(result,rule)
        else
            decision_tree(training_set,attribute,child_node,result)
        end
    end
end

decision_tree (generic function with 1 method)

## Hàm classify
***Dùng đế dán nhẫn "variety" cho một dòng dữ liệu***
## Input:
* X: một dòng dữ liệu trong tập test
* rules: tập luật có được từ hàm decision_tree
## Output:
* Nhãn thuộc "Setosa", "Versicolor", "Virginica"

In [123]:
function classify(X,rules)
    name = names(X)
    name = name[1:length(name)-1]
    for rule in rules
        flag = 0
        for (i,j) in rule
            if(i!="variety")
                if X[i]!=j
                    flag = 1
                    break
                end
            end
        end
        if(flag == 0)
            return rule["variety"]
        end
    end
    return "Versicolor"
end

classify (generic function with 1 method)

## Hàm fit
***Dùng để fit bộ test với các luật vừa tìm được***
## Input:
* test_set: tập test (1/3 dataset) đã rời rạc hoá
* rules: các luật vừa tìm được 
## Output:
* Độ đo accuracy
* variety: tập nhãn từ thuật toán (so tập này với thực tế xem đúng bao nhiêu phần trăm)

In [124]:
function fit(test_set, rules)
    variety = []
    for i in 1:size(test_set)[1]
        X = test_set[i,:]
        brand = classify(X,rules)
        push!(variety,brand)
    end
    test_variety = test_set[:,"variety"]
    count = 0
    for i in 1:length(test_variety)
        if(variety[i]==test_variety[i])
            count+=1
        end
    end
    return (count*100)/length(variety), variety
end

fit (generic function with 1 method)

In [125]:
df = CSV.File("iris.csv")
df = DataFrame(df)
train,test = splitdf(df,2/3)
training_set = regulazation(train)
test_set = regulazation(test)

,sepal.length,sepal.width,petal.length,petal.width,variety
,Float64,Float64,Float64,Float64,String15
1,2.0,2.0,1.0,2.0,Setosa
2,2.0,2.0,1.0,2.0,Setosa
3,2.0,2.0,1.0,2.0,Setosa
4,2.0,2.0,1.0,2.0,Setosa
5,2.0,2.0,1.0,2.0,Setosa
6,2.0,2.0,1.0,2.0,Setosa
7,2.0,2.0,1.0,2.0,Setosa
8,2.0,2.0,1.0,2.0,Setosa
9,2.0,2.0,1.0,2.0,Setosa


In [126]:
result = []
attribute = sort_by_entropy(training_set)
parent = Dict()
decision_tree(training_set,attribute,parent,result)
result

3-element Vector{Any}:
 Dict{Any, Any}("petal.length" => 1.0, "variety" => "Setosa")
 Dict{Any, Any}("petal.length" => 2.0, "petal.width" => 2.0, "sepal.length" => 1.0, "variety" => "Virginica")
 Dict{Any, Any}("petal.length" => 2.0, "petal.width" => 1.0, "variety" => "Virginica")

In [127]:
accuracy, variety = fit(test_set,result)

(92.0, Any["Setosa", "Setosa", "Setosa", "Setosa", "Setosa", "Setosa", "Setosa", "Setosa", "Setosa", "Setosa"  …  "Virginica", "Versicolor", "Virginica", "Virginica", "Virginica", "Versicolor", "Virginica", "Virginica", "Versicolor", "Virginica"])

In [128]:
predict = CSV.File("predict.csv")
predict = DataFrame(predict)

,sepal.length,sepal.width,petal.length,petal.width,variety
,Float64,Float64,Float64,Float64,String15
1,5.0,3.0,2.0,0.1,Versicolor
2,5.4,3.3,1.7,0.3,Versicolor
3,6.6,3.6,1.6,1.7,Versicolor
4,3.5,3.3,1.1,1.7,Versicolor
5,4.0,3.0,1.9,1.2,Versicolor
6,5.9,3.6,1.9,1.2,Versicolor
7,4.2,3.3,1.6,1.1,Versicolor
8,6.7,3.4,1.6,1.8,Versicolor
9,6.8,3.7,1.1,0.7,Versicolor


In [129]:
function Predict(data, rules)
    new = copy(data)
    new = regulazation(new)
    for i in 1:size(data)[1]
        class = classify(new[i,:],rules)
        data[i,"variety"] = class
    end
    return data
end

Predict (generic function with 1 method)

In [130]:
Predict(predict,result)

,sepal.length,sepal.width,petal.length,petal.width,variety
,Float64,Float64,Float64,Float64,String15
1,5.0,3.0,2.0,0.1,Setosa
2,5.4,3.3,1.7,0.3,Setosa
3,6.6,3.6,1.6,1.7,Setosa
4,3.5,3.3,1.1,1.7,Setosa
5,4.0,3.0,1.9,1.2,Setosa
6,5.9,3.6,1.9,1.2,Setosa
7,4.2,3.3,1.6,1.1,Setosa
8,6.7,3.4,1.6,1.8,Setosa
9,6.8,3.7,1.1,0.7,Setosa
